In [2]:
from datetime import datetime
import tensorflow as tf
import importlib
import numpy as np



In [3]:
# import config 
import configs.config
importlib.reload(configs.config)
from configs.config import CFG
from utils import config
config = config.Config.from_json(CFG)

## Create Dataset

In [4]:
# create Train Series and Dataset 
import dataloader.dataloader
importlib.reload(dataloader.dataloader)
from dataloader.dataloader import Series,Dataset
try: 
    del ds
except:
    pass
from dataloader.dataloader import Test_Series, Train_Series
try: 
    del test_series , train_series
except:
    pass

train_series = Train_Series(config.data,config.paths)
train_series.createSeries()
test_series = Test_Series(config.data,config.paths)
test_series.createSeries()
ds = Dataset(train_series,test_series)
ds.split_dataset()

X_train = ds.X_train
y_train = ds.y_train
X_test = ds.X_test
y_test = ds.y_test

print(X_train.shape)
print(X_test.shape)

(4405, 150, 5)
(45, 10, 5)


## Save Datset to Files 

In [5]:
# import pickle
# # Include the epoch in the file name (uses `str.format`)
# ds_path = "./model/saved_ds/ds.obj"

# with  open(ds_path,'wb') as file:
#     exampleObj = pickle.dump(ds,file)


In [8]:
import dill
ds_path = "./model/saved_ds/ds.pkl"
with open(ds_path,'wb') as file:
    my_pickle = dill.dump(ds,file)

TypeError: _Pickler.__init__() got an unexpected keyword argument 'ignore'

In [13]:
ds.__module__.__module__

AttributeError: 'str' object has no attribute '__module__'

## Save Test Series to Files 

In [ ]:
# Include the epoch in the file name (uses `str.format`)
test_path = "./model/saved_ds/test_series.obj"

with  open(test_path,'wb') as file:
    exampleObj = pickle.dump(ds,file)

## Call model and compile

In [31]:
from model.model1D import Unet,Unet_Model
import model.model1D
importlib.reload(model.model1D)
try: 
    del unet
except:
    pass

unet = Unet(n_classes=3,n_filters=8)
unet.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
unet.model((160,2)).summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 160, 2)]     0           []                               
                                                                                                  
 downsampling_block (Downsampli  ((None, 80, 8),     256         ['input_1[0][0]']                
 ngBlock)                        (None, 160, 8))                                                  
                                                                                                  
 downsampling_block_1 (Downsamp  ((None, 40, 16),    1184        ['downsampling_block[0][0]']     
 lingBlock)                      (None, 80, 16))                                                  
                                                                                              

## Train Model and save weights and model to files

In [8]:
from datetime import datetime
#train model
history = unet.fit(X_train,y_train,epochs = 100)




Epoch 1/100


ValueError: in user code:

    File "d:\DEV\Projekty\PythonProjects\venv\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "d:\DEV\Projekty\PythonProjects\venv\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\DEV\Projekty\PythonProjects\venv\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "d:\DEV\Projekty\PythonProjects\venv\lib\site-packages\keras\engine\training.py", line 1052, in train_step
        self._validate_target_and_loss(y, loss)
    File "d:\DEV\Projekty\PythonProjects\venv\lib\site-packages\keras\engine\training.py", line 1006, in _validate_target_and_loss
        raise ValueError(

    ValueError: Target data is missing. Your model was compiled with loss=<keras.losses.SparseCategoricalCrossentropy object at 0

In [ ]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "model/weights/cp-{epoch:04d}.ckpt"
# save weights 
unet.save_weights(checkpoint_path.format(epoch=100))
#save model
models_path = "model/saved_models/unet_{version}_{now}"
unet.save(models_path.format(version = "v1", now = str(datetime.now().date())))

In [32]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "model/weights/cp-{epoch:04d}.ckpt"

# Loads the weights
unet.load_weights(checkpoint_path.format(epoch =100))

# Re-evaluate the model
# loss, acc = unet.evaluate(test_images, test_labels, verbose=2)
# print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

In [50]:
# create Train Series and Dataset 
import dataloader.dataloader
importlib.reload(dataloader.dataloader)
from dataloader.dataloader import Test_Series
try: 
    del test_series
except:
    pass

test_series = Test_Series(config.data,config.paths)
test_series.createSeries()

In [54]:
import pandas as pd

In [69]:
def create_events(y_pred :np.array,serie):
    """create df_events 
    input:
    y_pred - score  (predicited vaues )"""
    def detectChange(last_val,current_val):
        if last_val == -1 or current_val == -1:
            return False

        return last_val !=current_val

    # create empty dataframe
    df_events = pd.DataFrame(columns=["series_id","step","event","score"])
    # unpad y_pred
    if serie.slice_pads is None:
        y_pred_unpadded = y_pred
    else:
        y_pred_unpadded = serie._unpad(y_pred,serie.slice_pads)
    # create step seg list
    #         - events - segmentation mask
    #         -  score   - predicited vaues for chosen event
    events = np.argmax(y_pred_unpadded,axis = -1,keepdims=True)
    score = np.max(y_pred_unpadded,axis=-1,keepdims=True)      
    for slice_num in range(events.shape[0]):
        for i in range(events.shape[-2]):
            event_val = events[slice_num,i][0]
            event_score = score[slice_num,i][0]
            if i == 0:
                # do not detect anything during first step
                continue
            elif not detectChange(events[slice_num,i-1],event_val):
                continue

            df_events.loc[len(df_events.index)] = [serie.serie_id,i,event_val,event_score]
    
    # decode events 
    print(df_events)
    df_events = serie.decode_events(df_events)
    # save as serie events 
    serie_events = df_events

    return serie_events 

In [70]:
def predict(model,test_serie):
    x = test_serie.slices
    y_pred = model.predict(x)
    serie_events = create_events(y_pred,test_serie)   
    return serie_events

test_series_pred = {}
for serie_id, test_serie in test_series.series.items():
    serie_events = predict(unet,test_serie)
    print(serie_events)


1/1 [==============================] - 0s 21ms/step


      series_id  step  event      score
0  038441c925bb     5      2  29.746996
1  038441c925bb    17      1  46.573971
2  038441c925bb    22      2  33.335999
3  038441c925bb    25      1  47.931240
      series_id  step   event      score
0  038441c925bb     5   onset  29.746996
1  038441c925bb    17  wakeup  46.573971
2  038441c925bb    22   onset  33.335999
3  038441c925bb    25  wakeup  47.931240
1/1 [==============================] - 0s 17ms/step
Empty DataFrame
Columns: [series_id, step, event, score]
Index: []
Empty DataFrame
Columns: [series_id, step, event, score]
Index: []


In [60]:
from matplotlib import pyplot as plt
plt.plot(history.history["accuracy"])

NameError: name 'history' is not defined

In [18]:
def accuracy(real,pred):
    comp = real == pred
    return np.sum(comp)/(comp.shape[0]*comp.shape[1])


y_train_pred = unet.predict(X_train)
print("train accuracy : " + str(accuracy(y_train,y_train_pred)))


y_test_pred = unet.predict(X_test)
print("test accuracy : " +  str(accuracy(y_test,y_test_pred)))

In [167]:
from IPython.display import FileLink
import pandas as pd

# import os
# os.chdir(r'kaggle/working')'
filename = 'train_' +'.csv'
pd.DataFrame(test_mask).to_csv(filename)

FileLink(filename)



ValueError: Must pass 2-d input. shape=(51, 8640, 3)